- MRR (Monthly Recurring Revenue)
- ARR (Annual Run Rate)
- TAS (Tasa de Adquisición de Suscriptores)
- LTV (Life Time Value)
- Churn rate
- Estatus de suscripción
- Más lo que decidas incluir

In [1]:
import streamlit as st
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('payments.xlsx',sheet_name='Hoja 1 - payments (2)',index_col=0, header=1)
df = df.reset_index().set_index('id')

In [3]:
#Converting to Datetime
df['paid_at'] = pd.to_datetime(df['paid_at'],infer_datetime_format=True)
df['failed_at'] = pd.to_datetime(df['failed_at'],infer_datetime_format=True)
df['created_at'] = pd.to_datetime(df['created_at'],infer_datetime_format=True)
df['updated_at'] = pd.to_datetime(df['updated_at'],infer_datetime_format=True)
df['deleted_at'] = pd.to_datetime(df['deleted_at'],infer_datetime_format=True)
df['required_action_at'] = pd.to_datetime(df['required_action_at'],infer_datetime_format=True)
df['refunded_at'] = pd.to_datetime(df['refunded_at'],infer_datetime_format=True)
df['attended_at'] = pd.to_datetime(df['attended_at'],infer_datetime_format=True)
df['refund_pending_at'] = pd.to_datetime(df['refund_pending_at'],infer_datetime_format=True)


In [4]:
df.head()

,payable_type,payable_id,status,paid_at,failed_at,failure_reason,amount,total,created_at,updated_at,...,refund_reason_id,refund_failed_at,refund_failed_reason,payer_type,payer_id,company_id,payment_gateway_fee_amount,attended_at,payments_failed,refund_pending_at
id,,,,,,,,,,,,,,,,,,,,,
44,App\Models\Companies\Subscription,67,paid,2021-02-26 15:36:54,NaT,NaN,9900,9139,2021-02-26 15:36:54,2021-04-15 12:14:08,...,NaN,NaN,NaN,App\Models\Clients\Client,59,9,761,NaT,NaN,NaT
45,App\Models\Companies\Subscription,63,paid,2021-02-26 16:11:52,NaT,NaN,3900,3390,2021-02-26 16:11:52,2021-04-15 12:14:08,...,NaN,NaN,NaN,App\Models\Clients\Client,56,9,510,NaT,NaN,NaT
39,App\Models\Companies\Subscription,58,paid,2021-02-19 12:40:08,NaT,NaN,9900,9139,2021-02-19 12:40:08,2021-04-15 12:14:08,...,NaN,NaN,NaN,App\Models\Clients\Client,53,9,761,NaT,NaN,NaT
46,App\Models\Companies\Subscription,68,failed,NaT,2021-02-26 20:45:48,Tarjeta rechazada.,3900,3390,2021-02-26 20:45:48,2021-04-15 12:14:09,...,NaN,NaN,NaN,App\Models\Clients\Client,57,9,510,NaT,NaN,NaT
40,App\Models\Companies\Subscription,61,paid,2021-02-23 15:55:20,NaT,NaN,3900,3390,2021-02-23 15:55:20,2021-04-15 12:14:08,...,NaN,NaN,NaN,App\Models\Clients\Client,55,9,510,NaT,NaN,NaT


In [5]:
# subscribed = df[df['status']=='paid']
# # subscribed = subscribed.resample('M',on='paid_at').sum()
# subscribed = subscribed.groupby(['company_id','paid_at'])['amount'].sum().reset_index()
# subscribed = subscribed.resample('M',on='paid_at')['amount'].mean().reset_index()
# subscribed

In [6]:
subscribed=df.groupby(['payer_id',pd.Grouper(key='paid_at',freq='M')])['amount'].sum()
subscribed

payer_id  paid_at   
10        2021-04-30    90000
53        2021-02-28     9900
          2021-03-31     9900
          2021-04-30     9900
55        2021-02-28     3900
                        ...  
234       2021-05-31     9900
235       2021-05-31     3900
236       2021-05-31    39900
241       2021-05-31    39900
242       2021-05-31    35000
Name: amount, Length: 173, dtype: int64

In [7]:
#Monthly Recurrent Revenue

MRR = subscribed.groupby('paid_at').sum().reset_index().rename(columns={'amount':'MRR','paid_at':'Date'})
MRR

,Date,MRR
0,2021-02-28,47400
1,2021-03-31,596500
2,2021-04-30,2978114
3,2021-05-31,1318900


In [8]:
# Annual Run Rate
ARR = MRR
ARR['MRR'] = MRR['MRR']*12
ARR= ARR.rename(columns={'MRR':'ARR'})
ARR

,Date,ARR
0,2021-02-28,568800
1,2021-03-31,7158000
2,2021-04-30,35737368
3,2021-05-31,15826800


In [9]:
# Average Revenue per User
ARPU = subscribed.groupby('paid_at').mean().reset_index().rename(columns={'amount':'ARPU','paid_at':'Date'})
ARPU['ARPU'] = ARPU['ARPU'].astype(int)
ARPU

,Date,ARPU
0,2021-02-28,7900
1,2021-03-31,19883
2,2021-04-30,32726
3,2021-05-31,28671


In [10]:
#Churn Rate
# Total # of Customers that Cancel / Total # of Customers (at given Month)

In [11]:
tot_custM =  df.groupby(['payer_id','paid_at'])['status'].count().reset_index()
tot_custM = tot_custM.resample('M',on='paid_at')['status'].count().reset_index().rename(columns={'status':'Total_users','paid_at':'Date'})
tot_custM 


,Date,Total_users
0,2021-02-28,6
1,2021-03-31,30
2,2021-04-30,92
3,2021-05-31,47


In [12]:
fail_custM = df[df['status']== 'failed']
fail_custM =  fail_custM.groupby(['payer_id','failed_at'])['status'].count().reset_index()
fail_custM = fail_custM.resample('M',on='failed_at')['status'].count().reset_index().rename(columns={'status':'Canceled','failed_at':'Date'})
fail_custM['Canceled'] = np.floor(fail_custM['Canceled']/4)
fail_custM = fail_custM[fail_custM['Canceled']>=1]
canceled = fail_custM
canceled['Canceled']= fail_custM['Canceled'].astype(int)
canceled

,Date,Canceled
1,2021-03-31,2
2,2021-04-30,4
3,2021-05-31,2


In [13]:
churn = canceled.merge(tot_custM, how='inner', on='Date')
churn['Churn_rate'] = churn.Canceled/churn.Total_users
churn['Months'] = 1/churn.Churn_rate
churn

,Date,Canceled,Total_users,Churn_rate,Months
0,2021-03-31,2,30,0.066667,15.0
1,2021-04-30,4,92,0.043478,23.0
2,2021-05-31,2,47,0.042553,23.5


In [14]:
#CLV Customer Life time Value
#ARPU*churn(months)
CLV = ARPU.merge(churn,how='inner',on='Date')
CLV['CLV'] = CLV['ARPU']*CLV['Months']
CLV['CLV'] = CLV['CLV'].astype(int)
CLV

,Date,ARPU,Canceled,Total_users,Churn_rate,Months,CLV
0,2021-03-31,19883,2,30,0.066667,15.0,298245
1,2021-04-30,32726,4,92,0.043478,23.0,752698
2,2021-05-31,28671,2,47,0.042553,23.5,673768


In [15]:
#CAS (TAS)
#Customer Aquisition Cost
